In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import optimizers, layers, Sequential, optimizers, regularizers, callbacks, losses, metrics
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [32]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.loc[:, 'filename'] = train_df['Id'] + '.jpg'

In [33]:
train_df

# CNN model

In [34]:
img_size = (200, 200, 3)
train_images_dir = '../input/petfinder-pawpularity-score/train'
image_gen_train = ImageDataGenerator(
    validation_split=0.20,
    rescale=1/255
)

train_gen = image_gen_train.flow_from_dataframe(
    directory=train_images_dir,
    target_size=img_size[:2],
    dataframe=train_df,
    x_col='filename',
    y_col='Pawpularity',
    class_mode='raw',
    seed=42,
    shuffle=True,
    subset='training'
)
val_gen = image_gen_train.flow_from_dataframe(
    directory=train_images_dir,
    target_size=img_size[:2],
    dataframe=train_df,
    x_col='filename',
    y_col='Pawpularity',
    class_mode='raw',
    seed=42,
    shuffle=True,
    subset='validation'
)

In [35]:
model = Sequential([
layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', input_shape=img_size),
layers.MaxPooling2D(2, 2),
layers.Dropout(0.2),
layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
layers.MaxPooling2D(2, 2),
layers.Dropout(0.2),
layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
layers.MaxPooling2D(2, 2),
layers.Dropout(0.2),
layers.Flatten(),
layers.Dense(128, activation='relu'),
layers.Dropout(0.5),
# layers.Dense(64, activation='relu'),
# layers.Dropout(0.5),
layers.Dense(1)
])
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss=losses.MeanSquaredError(), 
              metrics=[metrics.RootMeanSquaredError()])

In [36]:
model.summary()

In [37]:
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=1, patience=10)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=160,
#     steps_per_epoch=10,
#     use_multiprocessing=True
    callbacks=[es]
)

In [38]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [39]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [40]:
test_images_dir = '../input/petfinder-pawpularity-score/test'
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.loc[:, 'filename'] = test_df['Id'] + '.jpg'

image_gen_test = ImageDataGenerator(rescale=1/255.)
test_gen = image_gen_test.flow_from_dataframe(
    directory=test_images_dir,
    target_size=img_size[:2],
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    class_mode=None,
    shuffle=False,
)

In [41]:
preds = model.predict_generator(test_gen)
test_df['pred_1'] = list(x[0] for x in preds)

# XGB using tabular data

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [53]:
cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
X = train_df[cols]
y = train_df['Pawpularity']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

xgb_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.5, n_jobs = 4)
xgb_model.fit(X_train, y_train, early_stopping_rounds = 5, eval_set = [(X_val, y_val)], verbose = False)
preds_2 = xgb_model.predict(X_val)

mean_squared_error(preds_2, y_val, squared = False)

In [57]:
test_df['pred_2'] = xgb_model.predict(test_df[cols])

In [63]:
test_df['Pawpularity'] = (test_df['pred_1'] + test_df['pred_2'])/2

In [66]:
test_df[['Id', 'Pawpularity']].to_csv('submission.csv', index=True)